In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
# Device configuration
device = torch.device('cpu')

In [3]:
# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [4]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [5]:
# Image processing
# transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
#                                      std=(0.5, 0.5, 0.5))])
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

In [6]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

In [7]:
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [8]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

# Device setting
D = D.to(device)
G = G.to(device)

# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()
    
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        # 生成假的图片
        fake_images = G(z)
        # 训练假的图片
        outputs = D(fake_images)
        # 计算损失
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        # 两个损失加起来
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

Epoch [0/200], Step [200/600], d_loss: 0.0498, g_loss: 3.8937, D(x): 0.99, D(G(z)): 0.04
Epoch [0/200], Step [400/600], d_loss: 0.0566, g_loss: 5.7645, D(x): 0.98, D(G(z)): 0.03
Epoch [0/200], Step [600/600], d_loss: 0.1296, g_loss: 4.5314, D(x): 0.95, D(G(z)): 0.06
Epoch [1/200], Step [200/600], d_loss: 0.0371, g_loss: 4.6930, D(x): 1.00, D(G(z)): 0.03
Epoch [1/200], Step [400/600], d_loss: 0.1070, g_loss: 4.9126, D(x): 0.97, D(G(z)): 0.06
Epoch [1/200], Step [600/600], d_loss: 0.1849, g_loss: 4.4038, D(x): 0.97, D(G(z)): 0.14
Epoch [2/200], Step [200/600], d_loss: 0.2302, g_loss: 3.8912, D(x): 0.92, D(G(z)): 0.11
Epoch [2/200], Step [400/600], d_loss: 0.2251, g_loss: 4.2716, D(x): 0.92, D(G(z)): 0.08
Epoch [2/200], Step [600/600], d_loss: 0.1149, g_loss: 4.2831, D(x): 0.97, D(G(z)): 0.07
Epoch [3/200], Step [200/600], d_loss: 0.9470, g_loss: 3.7933, D(x): 0.91, D(G(z)): 0.49
Epoch [3/200], Step [400/600], d_loss: 0.3328, g_loss: 3.8729, D(x): 0.88, D(G(z)): 0.10
Epoch [3/200], Step [

Epoch [30/200], Step [600/600], d_loss: 0.2592, g_loss: 4.9510, D(x): 0.91, D(G(z)): 0.06
Epoch [31/200], Step [200/600], d_loss: 0.3227, g_loss: 4.2985, D(x): 0.87, D(G(z)): 0.04
Epoch [31/200], Step [400/600], d_loss: 0.2406, g_loss: 4.8154, D(x): 0.93, D(G(z)): 0.08
Epoch [31/200], Step [600/600], d_loss: 0.3586, g_loss: 3.3361, D(x): 0.83, D(G(z)): 0.05
Epoch [32/200], Step [200/600], d_loss: 0.4396, g_loss: 4.2002, D(x): 0.84, D(G(z)): 0.05
Epoch [32/200], Step [400/600], d_loss: 0.2716, g_loss: 3.3309, D(x): 0.91, D(G(z)): 0.09
Epoch [32/200], Step [600/600], d_loss: 0.2675, g_loss: 4.4866, D(x): 0.91, D(G(z)): 0.06
Epoch [33/200], Step [200/600], d_loss: 0.2480, g_loss: 4.1736, D(x): 0.92, D(G(z)): 0.09
Epoch [33/200], Step [400/600], d_loss: 0.4581, g_loss: 3.4968, D(x): 0.93, D(G(z)): 0.21
Epoch [33/200], Step [600/600], d_loss: 0.3030, g_loss: 4.0443, D(x): 0.91, D(G(z)): 0.12
Epoch [34/200], Step [200/600], d_loss: 0.4364, g_loss: 4.1541, D(x): 0.89, D(G(z)): 0.15
Epoch [34/

Epoch [61/200], Step [400/600], d_loss: 0.6613, g_loss: 2.7568, D(x): 0.76, D(G(z)): 0.18
Epoch [61/200], Step [600/600], d_loss: 0.6949, g_loss: 2.2560, D(x): 0.77, D(G(z)): 0.21
Epoch [62/200], Step [200/600], d_loss: 0.6337, g_loss: 2.2856, D(x): 0.86, D(G(z)): 0.26
Epoch [62/200], Step [400/600], d_loss: 0.7416, g_loss: 2.3798, D(x): 0.82, D(G(z)): 0.25
Epoch [62/200], Step [600/600], d_loss: 0.5935, g_loss: 2.4367, D(x): 0.83, D(G(z)): 0.21
Epoch [63/200], Step [200/600], d_loss: 0.6567, g_loss: 2.3876, D(x): 0.85, D(G(z)): 0.30
Epoch [63/200], Step [400/600], d_loss: 0.5988, g_loss: 2.8377, D(x): 0.76, D(G(z)): 0.15
Epoch [63/200], Step [600/600], d_loss: 0.5480, g_loss: 2.6770, D(x): 0.82, D(G(z)): 0.19
Epoch [64/200], Step [200/600], d_loss: 0.4740, g_loss: 2.8810, D(x): 0.82, D(G(z)): 0.14
Epoch [64/200], Step [400/600], d_loss: 0.5630, g_loss: 2.3659, D(x): 0.85, D(G(z)): 0.23
Epoch [64/200], Step [600/600], d_loss: 0.4268, g_loss: 3.2276, D(x): 0.82, D(G(z)): 0.10
Epoch [65/

Epoch [92/200], Step [200/600], d_loss: 0.8909, g_loss: 1.7558, D(x): 0.78, D(G(z)): 0.35
Epoch [92/200], Step [400/600], d_loss: 0.7234, g_loss: 2.0591, D(x): 0.75, D(G(z)): 0.26
Epoch [92/200], Step [600/600], d_loss: 0.8615, g_loss: 1.0777, D(x): 0.68, D(G(z)): 0.22
Epoch [93/200], Step [200/600], d_loss: 0.7210, g_loss: 2.0789, D(x): 0.76, D(G(z)): 0.24
Epoch [93/200], Step [400/600], d_loss: 0.7109, g_loss: 1.8033, D(x): 0.80, D(G(z)): 0.30
Epoch [93/200], Step [600/600], d_loss: 0.9298, g_loss: 2.0738, D(x): 0.64, D(G(z)): 0.15
Epoch [94/200], Step [200/600], d_loss: 0.8676, g_loss: 1.9294, D(x): 0.71, D(G(z)): 0.26
Epoch [94/200], Step [400/600], d_loss: 0.8819, g_loss: 1.5975, D(x): 0.75, D(G(z)): 0.30
Epoch [94/200], Step [600/600], d_loss: 0.8730, g_loss: 1.9996, D(x): 0.68, D(G(z)): 0.24
Epoch [95/200], Step [200/600], d_loss: 0.7635, g_loss: 1.4946, D(x): 0.80, D(G(z)): 0.31
Epoch [95/200], Step [400/600], d_loss: 0.7634, g_loss: 1.7181, D(x): 0.75, D(G(z)): 0.28
Epoch [95/

Epoch [122/200], Step [400/600], d_loss: 0.7745, g_loss: 2.5985, D(x): 0.69, D(G(z)): 0.19
Epoch [122/200], Step [600/600], d_loss: 0.8984, g_loss: 1.4141, D(x): 0.69, D(G(z)): 0.28
Epoch [123/200], Step [200/600], d_loss: 1.0243, g_loss: 1.5146, D(x): 0.70, D(G(z)): 0.31
Epoch [123/200], Step [400/600], d_loss: 0.8079, g_loss: 2.0393, D(x): 0.77, D(G(z)): 0.30
Epoch [123/200], Step [600/600], d_loss: 0.6672, g_loss: 1.8303, D(x): 0.80, D(G(z)): 0.24
Epoch [124/200], Step [200/600], d_loss: 0.8876, g_loss: 2.0289, D(x): 0.72, D(G(z)): 0.31
Epoch [124/200], Step [400/600], d_loss: 1.0528, g_loss: 1.5731, D(x): 0.62, D(G(z)): 0.23
Epoch [124/200], Step [600/600], d_loss: 0.9924, g_loss: 1.7484, D(x): 0.65, D(G(z)): 0.26
Epoch [125/200], Step [200/600], d_loss: 0.9547, g_loss: 1.4825, D(x): 0.73, D(G(z)): 0.35
Epoch [125/200], Step [400/600], d_loss: 0.8272, g_loss: 1.6970, D(x): 0.79, D(G(z)): 0.36
Epoch [125/200], Step [600/600], d_loss: 0.9622, g_loss: 1.9162, D(x): 0.62, D(G(z)): 0.23

Epoch [152/200], Step [600/600], d_loss: 1.0581, g_loss: 1.5135, D(x): 0.70, D(G(z)): 0.38
Epoch [153/200], Step [200/600], d_loss: 0.8109, g_loss: 1.5275, D(x): 0.71, D(G(z)): 0.26
Epoch [153/200], Step [400/600], d_loss: 0.8068, g_loss: 1.8904, D(x): 0.68, D(G(z)): 0.19
Epoch [153/200], Step [600/600], d_loss: 0.7708, g_loss: 1.6838, D(x): 0.78, D(G(z)): 0.31
Epoch [154/200], Step [200/600], d_loss: 1.0495, g_loss: 1.5856, D(x): 0.65, D(G(z)): 0.28
Epoch [154/200], Step [400/600], d_loss: 1.0614, g_loss: 1.3772, D(x): 0.64, D(G(z)): 0.31
Epoch [154/200], Step [600/600], d_loss: 0.8595, g_loss: 1.5421, D(x): 0.76, D(G(z)): 0.34
Epoch [155/200], Step [200/600], d_loss: 0.8690, g_loss: 1.5940, D(x): 0.79, D(G(z)): 0.37
Epoch [155/200], Step [400/600], d_loss: 1.1303, g_loss: 1.3644, D(x): 0.71, D(G(z)): 0.44
Epoch [155/200], Step [600/600], d_loss: 0.9630, g_loss: 1.8906, D(x): 0.69, D(G(z)): 0.31
Epoch [156/200], Step [200/600], d_loss: 0.9747, g_loss: 1.4681, D(x): 0.65, D(G(z)): 0.27

Epoch [183/200], Step [200/600], d_loss: 1.0582, g_loss: 1.2926, D(x): 0.66, D(G(z)): 0.34
Epoch [183/200], Step [400/600], d_loss: 1.0951, g_loss: 1.6295, D(x): 0.61, D(G(z)): 0.29
Epoch [183/200], Step [600/600], d_loss: 1.0807, g_loss: 1.2514, D(x): 0.63, D(G(z)): 0.32
Epoch [184/200], Step [200/600], d_loss: 1.1860, g_loss: 1.6122, D(x): 0.63, D(G(z)): 0.33
Epoch [184/200], Step [400/600], d_loss: 0.9830, g_loss: 1.6750, D(x): 0.67, D(G(z)): 0.30
Epoch [184/200], Step [600/600], d_loss: 0.8963, g_loss: 1.7260, D(x): 0.64, D(G(z)): 0.23
Epoch [185/200], Step [200/600], d_loss: 1.0068, g_loss: 1.7142, D(x): 0.59, D(G(z)): 0.24
Epoch [185/200], Step [400/600], d_loss: 1.0461, g_loss: 1.5131, D(x): 0.59, D(G(z)): 0.27
Epoch [185/200], Step [600/600], d_loss: 0.8637, g_loss: 1.6021, D(x): 0.76, D(G(z)): 0.36
Epoch [186/200], Step [200/600], d_loss: 0.9450, g_loss: 1.2779, D(x): 0.78, D(G(z)): 0.38
Epoch [186/200], Step [400/600], d_loss: 0.8693, g_loss: 1.5893, D(x): 0.66, D(G(z)): 0.23